# Download images code
From: https://github.com/pereira-gha/activefire/

Images taken from satellite Landsat-8 and #poner la segunda api y de donde saca las imagenes#

In [1]:
#Library import
import os
import time
import pandas as pd
import rasterio

In [2]:
#Variables creation

#Link to the api of the Landsat 8 images
GC_L8 = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/'

MTL_EXTENSION = '_MTL.txt'
ext = '.TIF'

IN_DIR = r'../images/landsat_images/'
OUT_DIR = r'../images/landsat_images/tiff_images/'

if not os.path.exists(OUT_DIR): 
        os.makedirs(OUT_DIR) 

csv = "images_raster_Gianca.csv" #### Cambiar por el csv de cada quien
df = pd.read_csv(os.path.join(IN_DIR, csv), sep=';', index_col=0)
df.head()

,productId,entityId,acquisitionDate,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url,PR,tipo
2073096,LC08_L1TP_118016_20200901_20200901_01_RT,LC81180162020245LGN00,2020-09-01 02:16:24.269623,43.82,L1TP,118,16,61.70824,132.79747,63.95174,137.63490,https://s3-us-west-2.amazonaws.com/landsat-pds...,118016,RT
2073103,LC08_L1TP_118017_20200901_20200901_01_RT,LC81180172020245LGN00,2020-09-01 02:16:48.152191,57.11,L1TP,118,17,60.32189,131.83264,62.55944,136.53466,https://s3-us-west-2.amazonaws.com/landsat-pds...,118017,RT
2073176,LC08_L1TP_134039_20200901_20200901_01_RT,LC81340392020245LGN00,2020-09-01 04:04:26.713446,14.99,L1TP,134,39,29.20954,95.59589,31.34188,98.06246,https://s3-us-west-2.amazonaws.com/landsat-pds...,134039,RT
2073382,LC08_L1TP_093086_20200902_20200902_01_RT,LC80930862020246LGN00,2020-09-02 00:09:48.053934,65.78,L1TP,93,86,-38.54002,143.01550,-36.40002,145.64833,https://s3-us-west-2.amazonaws.com/landsat-pds...,93086,RT
2073595,LC08_L1TP_045032_20200901_20200901_01_RT,LC80450322020245LGN00,2020-09-01 18:51:37.359774,13.64,L1TP,45,32,39.25627,-124.14699,41.38561,-121.42375,https://s3-us-west-2.amazonaws.com/landsat-pds...,45032,RT


In [3]:
df.describe()

,cloudCover,path,row,min_lat,min_lon,max_lat,max_lon,PR
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,30.049474,115.684211,36.973684,31.525285,11.070328,33.730332,15.521804,115721.184211
std,31.151343,73.999858,27.459533,38.967863,95.432527,39.046780,95.524812,74012.886296
min,0.000000,3.000000,1.000000,-44.304530,-125.306560,-42.127910,-122.455630,3005.000000
25%,4.020000,46.000000,16.000000,-12.995808,-63.919477,-10.882542,-61.582667,46021.000000
50%,16.990000,118.000000,29.500000,42.806090,7.732400,44.951880,10.669880,118016.000000
75%,50.122500,179.000000,68.250000,61.679145,129.689832,63.933570,133.014645,179053.000000
max,100.000000,229.000000,90.000000,79.451430,144.829260,81.914190,147.228320,229090.000000


In [4]:
for ind, image_name in enumerate(df.productId.to_list()):
    start_time = time.time()
    print(ind+1, image_name)
    
    log = OUT_DIR + image_name + '.log'  
    if not os.path.exists(log):
        try:               
            path = image_name[10:13]
            row = image_name[13:16]      
            
            ########################### BQA ################################
            outnameBQA = OUT_DIR + image_name + '_' + 'BQA' + ext 
            if not os.path.exists(outnameBQA):
                link = GC_L8 + path + '/' + row + '/' + image_name + '/' + image_name + '_' + 'BQA' + ext
                with rasterio.open(link) as src:
                    profile = src.profile.copy()
                    BQA = src.read(1)
                
                with rasterio.open(outnameBQA, 'w', **profile) as dst:
                    dst.write_band(1, BQA.astype(rasterio.uint16))
                                   
            ########################### BANDS ################################
            outname = OUT_DIR + image_name + ext   
            if not os.path.exists(outname):
                link_bands = [GC_L8 + path + '/' + row + '/' + image_name + '/' + image_name + '_' + f'B{i}' + ext 
                            for i in [1,2,3,4,5,6,7,9,10,11]]
                
                bands = []
                profile = None
                for link in link_bands:
                    with rasterio.open(link) as src:
                        bands.append(src.read(1))
                        if profile is None:
                            profile = src.profile.copy()

                profile.update({
                    'dtype': rasterio.uint16,
                    'height': bands[0].shape[0],
                    'width': bands[0].shape[1],
                    'count': len(bands)
                })
                
                with rasterio.open(outname, 'w', **profile) as dst:
                    for i, band in enumerate(bands):
                        dst.write_band(i+1, band.astype(rasterio.uint16))

            # Move log file creation inside try block
            with open(log, 'w') as f:
                f.write('Done!')
            
            end_time = time.time()    
            print(f'Elapsed: {round((end_time-start_time)/60, 2)} min.\n')
            
        except Exception as e:
            print(f"Error processing {image_name}: {str(e)}")
            if os.path.exists(log):
                os.remove(log)  # Remove log file if processing failed

1 LC08_L1TP_118016_20200901_20200901_01_RT
2 LC08_L1TP_118017_20200901_20200901_01_RT
3 LC08_L1TP_134039_20200901_20200901_01_RT
4 LC08_L1TP_093086_20200902_20200902_01_RT
5 LC08_L1TP_045032_20200901_20200901_01_RT
6 LC08_L1GT_157012_20200902_20200902_01_RT
7 LC08_L1TP_007003_20200907_20200907_01_RT
8 LC08_L1TP_025033_20200905_20200905_01_RT
9 LC08_L1TP_041035_20200905_20200905_01_RT
10 LC08_L1TP_046021_20200908_20200908_01_RT
11 LC08_L1TP_046029_20200908_20200908_01_RT
12 LC08_L1TP_046031_20200908_20200908_01_RT
13 LC08_L1TP_092086_20200911_20200911_01_RT
14 LC08_L1TP_105069_20200906_20200906_01_RT
15 LC08_L1TP_117016_20200910_20200910_01_RT
16 LC08_L1TP_117017_20200910_20200910_01_RT
17 LC08_L1TP_119016_20200908_20200908_01_RT
18 LC08_L1TP_119017_20200908_20200908_01_RT
Elapsed: 4.76 min.

19 LC08_L1GT_156012_20200911_20200911_01_RT


KeyboardInterrupt: 